In [1]:
# import library
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
#function mengubah nama mata kuliah yang ekivalensi dari kurikulum 2016 ke 2020
def ubahNamaMK(df):
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'LOGIKA MATEMATIKA A', value = 'LOGIKA MATEMATIKA')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'KALKULUS IB', value = 'KALKULUS')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'LITERASI TIK', value = 'STATISTIKA')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'MATEMATIKA DISKRIT A', value = 'MATEMATIKA DISKRIT')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'DASAR ALGORITMA DAN PEMROGRAMAN', value = 'ALGORITMA PEMROGRAMAN')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'KALKULUS IIB', value = 'KALKULUS LANJUT')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'DESAIN DAN ANALISIS ALGORITMA', value = 'ANALISIS KOMPLEKSITAS ALGORITMA')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'ANALISIS DAN PERANCANGAN PERANGKAT LUNAK', value = 'RPL: ANALISIS KEBUTUHAN')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'PROBABILITAS DAN STATISTIKA A', value = 'TEORI PELUANG')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'KECERDASAN BUATAN', value = 'PENGANTAR KECERDASAN BUATAN')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'SISTEM OPERASI A', value = 'SISTEM OPERASI')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'TEORI BAHASA & AUTOMATA', value = 'TEORI BAHASA DAN AUTOMATA')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'INTERAKSI MANUSIA DAN KOMPUTER', value = 'INTERAKSI MANUSIA KOMPUTER')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'INTERAKSI MANUSIA DAN KOMPUTER A', value = 'INTERAKSI MANUSIA KOMPUTER')
    df['MATAKULIAH'] = df['MATAKULIAH'].replace(to_replace = 'PEMROGRAMAN BERORIENTASI OBYEK A', value = 'PEMROGRAMAN BERORIENTASI OBJEK')
    
#function membuat dataset baru dengan nama mata kuliah sebagai kolomnya
def datasetBaru(df):
    #menentukan nama file (pada folder csv) berdasarkan angkatan
    if df['ANGKATAN'][0] == 1617:
        file1 = 'csv/list_mk_angkatan16.csv'
        file2 = 'csv/data_baru1617.csv'
    elif df['ANGKATAN'][0] == 1718:
        file1 = 'csv/list_mk_angkatan17.csv'
        file2 = 'csv/data_baru1718.csv'
    elif df['ANGKATAN'][0] == 1819:
        file1 = 'csv/list_mk_angkatan18.csv'
        file2 = 'csv/data_baru1819.csv'
    
    #melakukan drop pada beberapa kolom
    df = df.drop(['NIM','ANGKATAN','INDEXPOINT'], axis = 1)
    #menghapus data yang duplicate
    df = df.drop_duplicates('MATAKULIAH', keep = 'first')
    df = df.reset_index(drop=True)
    
    #menyimpan file yg berisi KODE_MK dan MATAKULIAH
    df.to_csv(file1, index = False)
    
    #mengubah kolom KODE_MK menjadi list
    list_mk = df['MATAKULIAH'].tolist()
    df_fix = pd.DataFrame()

    #membuat kolom pada dataset baru (df_fix)
    df_fix['NIM'] = np.nan
    df_fix[list_mk] = np.nan
    df_fix.to_csv(file2, index = False)

#function mengubah index nilai alfabet menjadi numerik
def encoderNilai(df):
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'A', value = 4)
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'AB', value = 3.5)
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'B', value = 3)
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'BC', value = 2.5)
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'C', value = 2)
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'D', value = 1)
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'E', value = 0)
    df['INDEXPOINT'] = df['INDEXPOINT'].replace(to_replace = 'T', value = 0)

#function pemindahan dari dataset lama ke dataset baru
def pindahDataseBaru(df,df_fix):
    #membuat baris ke-0 pada dataframe_fix
    df_fix.loc[0] = np.nan
    df_fix['NIM'][0] = df['NIM'][0]
    
    i = 0
    #melakukan perulangan untuk memasukkan nilai dari dataset lama ke dataset baru
    for index, row in df.iterrows():
        if (df_fix['NIM'] == df['NIM'][index]).any() == True:
            #perulangan untuk memasukkan nilai sesuai dengan kolom pada df_fix
            for column in df_fix:
                if df['MATAKULIAH'][index] == column:
                    #kondisi jika nilai MK yg akan dimasukkan tidak kosong (mengulang mata kuliah)
                    if np.isnan(df_fix[column][i]) == True:
                        nilai = df.loc[index, 'INDEXPOINT']
                        df_fix[column][i] = nilai
                    elif np.isnan(df_fix[column][i]) == False:
                        nilai1 = df.loc[index, 'INDEXPOINT']
                        nilai2 = df_fix[column][i]
                        if nilai2 < nilai1:
                            df_fix[column][i] = nilai1          
        else:
            i = i + 1
            df_fix.loc[i] = np.nan
            df_fix['NIM'][i] = df['NIM'][index]
            #perulangan untuk memasukkan nilai sesuai dengan kolom pada df_fix
            for column in df_fix:
                if df['MATAKULIAH'][index] == column:
                    nilai = df.loc[index, 'INDEXPOINT']
                    df_fix[column][i] = nilai
    return df_fix

In [3]:
#melakukan import isi data_1617.csv dan membuat file list mata kuliah pada angkatan 2016
df = pd.read_csv("csv/data_1617.csv")
ubahNamaMK(df)
datasetBaru(df)

In [4]:
#melakukan import isi data_baru1617.csv, melakukan encoderNilai dan pemindahan data ke dataset baru
df_fix = pd.read_csv("csv/data_baru1617.csv")
encoderNilai(df)
df_fix = pindahDataseBaru(df,df_fix)

In [5]:
#mengisi data null menjadi 0 (mahasiswa yg tidak mengambil suatu mkpil atau belum mengambil)
df_fix = df_fix.fillna(0)
df_fix.to_csv('csv/data_fix1617.csv', index = False)
df_fix

,NIM,TEORI BAHASA DAN AUTOMATA,INTERAKSI MANUSIA KOMPUTER,TATA TULIS ILMIAH DAN TEKNIK PRESENTASI,IMPLEMENTASI DAN PENGUJIAN PERANGKAT LUNAK,ANALISIS KOMPLEKSITAS ALGORITMA,PENULISAN PROPOSAL,PEMODELAN BASIS DATA,FORENSIK KOMPUTER DAN JARINGAN,KALKULUS LANJUT,...,WRAP-ENTREPRENEURSHIP-BTP 4: STRATEGI PEMASARAN,MAGANG-DTI,IDENTIFIKASI MASALAH PADA SAINS DATA,IMPLEMENTASI SOLUSI PADA SISTEM CERDAS,KOMPUTASI LUNAK,METODOLOGI PENELITIAN,KOMPUTASI SOSIAL,KOMPRESI DATA,MODEL GRAF PROBABILISTIK,WRAP-ENTREPRENEURSHIP-BTP 2: VALIDASI IDE
0,sfff91bad0515fc6,4.0,3.5,4.0,4.0,1.0,4.0,3.5,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,sfeed7b028f982ab,2.0,3.5,4.0,4.0,4.0,3.5,3.5,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,sfeed40fe1e91f54,0.0,3.0,1.0,0.0,1.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sfeebfab53074ba2,4.0,3.5,4.0,3.5,2.0,3.0,3.5,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sfe99832907c34ae,3.5,3.0,4.0,2.5,1.0,3.0,3.5,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,s009888969c44b60,2.5,3.0,3.5,4.0,2.0,3.5,3.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
803,s007d10d856c2678,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
804,s005d6f1ca63593e,3.5,3.5,4.0,3.5,2.5,4.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
805,s003a586461c1d8a,2.0,3.5,3.5,4.0,3.0,3.0,3.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#melakukan import isi data_1718.csv, mengubah ubahNamaMK dan membuat datasetBaru
df = pd.read_csv("csv/data_1718.csv")
ubahNamaMK(df)
datasetBaru(df)

In [7]:
#melakukan import isi data_baru1718.csv, melakukan encoderNilai dan pemindahan data ke dataset baru
df_fix = pd.read_csv("csv/data_baru1718.csv")
encoderNilai(df)
df_fix = pindahDataseBaru(df,df_fix)

In [8]:
#mengisi data null menjadi 0 (mahasiswa yg tidak mengambil suatu mkpil atau belum mengambil)
df_fix = df_fix.fillna(0)
df_fix.to_csv('csv/data_fix1718.csv', index = False)
df_fix

,NIM,SISTEM PARALEL DAN TERDISTRIBUSI,MATRIKS DAN RUANG VEKTOR,KALKULUS LANJUT,TEORI BAHASA DAN AUTOMATA,SISTEM BASIS DATA,STRUKTUR DATA,ORGANISASI DAN ARSITEKTUR KOMPUTER,PEMBELAJARAN MESIN,PANCASILA DAN KEWARGANEGARAAN,...,PEMBENTUKAN KARAKTER,PERANCANGAN SOLUSI PADA SAINS DATA,IMPLEMENTASI SOLUSI PADA SAINS DATA,TOPIK KHUSUS 3 MCE,DATA SAINS UNTUK KOMPETISI,PENDIDIKAN AGAMA BUDHA DAN ETIKA,TOPIK PILIHAN PADA SISTEM PERANGKAT LUNAK LANJUT,SISTEM PERANGKAT LUNAK LANJUT II,SISTEM PERANGKAT LUNAK LANJUT III,SISTEM PERANGKAT LUNAK LANJUT I
0,sfe67e51b4dbcd95,4.0,4.0,3.5,3.5,2.5,4.0,4.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,0
1,sfe67825eb1ec669,4.0,3.5,3.0,3.0,2.0,2.5,3.0,3.0,3.5,...,0,0,0,0,0,0,0,0,0,0
2,sfe499de9af9c82b,3.5,1.0,2.0,2.0,1.0,2.0,2.0,3.0,3.5,...,0,0,0,0,0,0,0,0,0,0
3,sfe00aaa357a512f,4.0,2.0,3.0,3.0,3.5,2.0,2.5,4.0,4.0,...,0,0,0,0,0,0,0,0,0,0
4,sfd91fb05b6c8f7a,3.0,1.0,1.0,1.0,2.5,2.0,2.5,2.5,3.5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,s0151d455fde4fe8,4.0,4.0,4.0,2.5,2.5,2.5,3.5,3.5,4.0,...,0,0,0,0,0,0,0,0,0,0
769,s013d1b038d269d6,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0
770,s00adab89ebc7558,4.0,2.5,2.5,3.5,3.0,2.5,2.5,3.5,4.0,...,0,0,0,0,0,0,0,0,0,0
771,s009e63f36796e76,4.0,2.5,3.0,3.5,3.0,2.0,3.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#melakukan import isi data_1819.csv, mengubah ubahNamaMK dan membuat datasetBaru
df = pd.read_csv("csv/data_1819.csv")
ubahNamaMK(df)
datasetBaru(df)

In [10]:
#melakukan import isi data_baru1819.csv, melakukan encoderNilai dan pemindahan data ke dataset baru
df_fix = pd.read_csv("csv/data_baru1819.csv")
encoderNilai(df)
df_fix = pindahDataseBaru(df,df_fix)

In [11]:
#mengisi data null menjadi 0 (mahasiswa yg tidak mengambil suatu mkpil atau belum mengambil)
df_fix = df_fix.fillna(0)
df_fix.to_csv('csv/data_fix1819.csv', index = False)
df_fix

,NIM,KALKULUS LANJUT,JARINGAN KOMPUTER,STRUKTUR DATA,MATRIKS DAN RUANG VEKTOR,SISTEM DIGITAL,RPL: ANALISIS KEBUTUHAN,BAHASA INDONESIA,ORGANISASI DAN ARSITEKTUR KOMPUTER,KALKULUS,...,IDENTIFIKASI MASALAH PADA CYBER PHYSICAL SYSTEM,PENDIDIKAN KEWARGANEGARAAN,KOMPUTASI FINANSIAL,IMPLEMENTASI SOLUSI PADA SISTEM CERDAS,SOFTWARE DEFINED NETWORK,WRAP RESEARCHSHIP NLP-TM1: STUDI DATASET DAN SOURCE CODE,WRAP RESEARCHSHIP NLP-TM2: STUDI LITERATUR 1,IMPLEMENTASI SOLUSI PADA SAINS DATA,PEMULIHAN BENCANA,DESAIN ANTAR MUKA APLIKASI
0,sff786098612178a,0.0,0.0,2.0,0.0,3.0,0.0,3.5,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,sfeef8dba4584021,4.0,3.0,3.0,3.5,4.0,4.0,4.0,3.5,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,sfe3a916a6bf57bd,1.0,3.0,3.0,2.5,3.5,4.0,3.5,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sfe298ac57c318fe,1.0,4.0,2.0,3.5,3.5,2.5,3.0,3.5,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sfe11d7815e00d77,4.0,3.5,2.5,4.0,3.5,3.5,4.0,3.5,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,s01d2bd8ea2f6766,4.0,4.0,2.0,2.0,4.0,2.5,2.5,3.5,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
628,s01a0dc3f95e875a,1.0,4.0,4.0,1.0,4.0,1.0,3.5,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
629,s016632d0d44475c,3.5,2.5,2.0,2.0,3.0,2.0,3.5,2.5,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
630,s014ebcc7959812c,1.0,4.0,3.0,1.0,3.5,4.0,3.5,3.5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
